In [4]:
import argparse

import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from nltk.corpus import stopwords

from models import Concat, Discriminator, Generator
from dataset import Birds


stopwords = set(stopwords.words('english'))

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
class demoData(Dataset):
    
    def __init__(self):
        self.bird = Birds()
    
    def __len__(self):
        return 1

    def __getitem__(self,i):
        embedding_avg = np.empty(self.bird.encoding_dim)
        for word in self.sen.split():
            if word in stopwords or word not in self.bird.embeddings:
                continue
            embedding_avg += self.bird.embeddings[word]
        embedding_avg /= len(sen.split())
        real_embedding = torch.from_numpy(embedding_avg)
        return real_embedding.double()
    
    def input(self, sen):
        self.sen =  sen

In [5]:
demosen = demoData()

Loading Birds dataset in `dataset.py`: descriptions_per_img=10, encoding_dim=1024, incl_stopwords=True
Loading images...
done!
Loading txt descriptions...
done!
Training word embeddings...
done!
Creating text encodings...
done!


In [30]:
#opt = parser.parse_args()

device = torch.device("cpu")
ngpu = 1 # num gpus
nz = 100

sen = "a bird with blue body and red tail"
real_embedding = None
demosen.input(sen)

for i in range(len(demosen)):
    real_embedding = demosen[i]

netG = Generator(ngpu)
netG.apply(weights_init)
netG.load_state_dict(torch.load("netG_epoch_49.pth", map_location='cpu'))

optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

noise = torch.randn(64, nz, 1, 1, device=device)
#fake = netG(real_embedding, noise)
fixed_noise = torch.randn(128, nz, 1, 1, device=device)
print(real_embedding.float().dim())
print(fixed_noise.float().dim())
fake = netG(real_embedding.unsqueeze(2).unsqueeze(3),fixed_noise.float().unsqueeze(2).unsqueeze(3))
vutils.save_image(fake.detach(),
        '%s/fake_samples_epoch_%03d.png' % ('./Results', epoch),
        normalize=True)
print("done")

1
4


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


RuntimeError: Dimension out of range (expected to be in range of [-2, 1], but got 2)